In [1]:
import sempy.fabric as fabric

notebooks = fabric.list_items().query("Type == 'Notebook'")
notebooks

StatementMeta(, e3a40f80-27ed-4b26-9292-1b0ff830f99c, 3, Finished, Available, Finished)

,Id,Display Name,Description,Type,Workspace Id
12,21dce765-96a1-45a5-88b1-451713256b04,NB_bronze_get_data_posts,New notebook,Notebook,f9fdd80e-6849-4e0c-b57d-49061c7c36af
13,36a9be5b-d176-478c-aa04-628cbf74ce9d,NB_bronze_get_data_users,New notebook,Notebook,f9fdd80e-6849-4e0c-b57d-49061c7c36af
14,4a5bc05d-95db-4021-8761-cac67c7e8e4e,NB_bronze_get_data_tags,New notebook,Notebook,f9fdd80e-6849-4e0c-b57d-49061c7c36af
15,8deff75c-ca21-456b-bba1-922c31de5593,NB_silver_create_dim_date,New notebook,Notebook,f9fdd80e-6849-4e0c-b57d-49061c7c36af
16,8e0bb446-64bf-4e1e-bad3-c5a70d342a88,NB_silver_clean_data,New notebook,Notebook,f9fdd80e-6849-4e0c-b57d-49061c7c36af
17,e5fcc2c7-047e-4706-9b4d-09acc389be37,NB_gold_structured_data,New notebook,Notebook,f9fdd80e-6849-4e0c-b57d-49061c7c36af
19,7477dcc5-5dcd-4758-9249-ab87debb4915,NB_DAG,New notebook,Notebook,f9fdd80e-6849-4e0c-b57d-49061c7c36af
21,465ba7a4-fa25-444e-bb02-9c409a9283d2,NB_Credentials,New notebook,Notebook,f9fdd80e-6849-4e0c-b57d-49061c7c36af


In [2]:
import notebookutils.notebook as notebook

# Definir el DAG para la ejecución en secuencia con `useRootDefaultLakehouse`
DAG = {
    "activities": [
        {
            "name": "NB_bronze_get_data_posts",
            "path": "NB_bronze_get_data_posts",
            "timeoutPerCellInSeconds": 300,
            "args": {"useRootDefaultLakehouse": True}
        },
        {
            "name": "NB_bronze_get_data_users",
            "path": "NB_bronze_get_data_users",
            "timeoutPerCellInSeconds": 300,
            "dependencies": ["NB_bronze_get_data_posts"],
            "args": {"useRootDefaultLakehouse": True}
        },
        {
            "name": "NB_bronze_get_data_tags",
            "path": "NB_bronze_get_data_tags",
            "timeoutPerCellInSeconds": 300,
            "dependencies": ["NB_bronze_get_data_posts"],
            "args": {"useRootDefaultLakehouse": True}
        },
        {
            "name": "NB_silver_clean_data",
            "path": "NB_silver_clean_data",
            "timeoutPerCellInSeconds": 600,
            "dependencies": [
                "NB_bronze_get_data_users",
                "NB_bronze_get_data_tags",
                "NB_bronze_get_data_posts"
            ],
            "args": {"useRootDefaultLakehouse": True}
        },
        {
            "name": "NB_gold_structured_data",
            "path": "NB_gold_structured_data",
            "timeoutPerCellInSeconds": 600,
            "dependencies": ["NB_silver_clean_data"],
            "args": {"useRootDefaultLakehouse": True}
        }
    ],
    "timeoutInSeconds": 3600,  # Tiempo máximo para la ejecución del pipeline (1 hora)
    "concurrency": 5  # Ejecutar hasta 3 notebooks en paralelo cuando sea posible
}

# Ejecutar y visualizar el DAG con manejo de errores
try:
    result = notebook.runMultiple(DAG, {"displayDAGViaGraphviz": True, "DAGLayout": "spectral", "DAGSize": 11})
    print("✅ DAG ejecutado con éxito.")
except notebookutils.mssparkutils.handlers.notebookHandler.RunMultipleFailedException as e:
    print("❌ Error en la ejecución del DAG:")
    
    # Obtener información de los notebooks fallidos
    failed_notebooks = []
    if hasattr(e, 'result'):
        for activity, details in e.result.items():
            if details.get("status") != "Succeeded":
                failed_notebooks.append(f"📌 {activity} → Estado: {details.get('status')}, Error: {details.get('errorMessage', 'Desconocido')}")
    
    # Mostrar los notebooks que fallaron
    if failed_notebooks:
        print("\n🔴 Notebooks con errores:")
        for nb in failed_notebooks:
            print(nb)
    else:
        print("No se pudo obtener información detallada de los fallos.")

    raise e  # Opcional: Relanzar el error para que el DAG se detenga por completo


StatementMeta(, e3a40f80-27ed-4b26-9292-1b0ff830f99c, 4, Finished, Available, Finished)

✅ DAG ejecutado con éxito.
